# Preprost klasifikator slik

Ta zvezek vam pokaže, kako razvrščati slike z uporabo vnaprej naučenega nevronskega omrežja.

**Kaj boste naučili:**
- Kako naložiti in uporabiti vnaprej naučen model
- Predobdelava slik
- Napovedovanje na podlagi slik
- Razumevanje stopenj zaupanja

**Primer uporabe:** Prepoznavanje predmetov na slikah (kot so "mačka", "pes", "avto" itd.)

---


## Korak 1: Uvoz potrebnih knjižnic

Uvozimo orodja, ki jih potrebujemo. Ne skrbite, če jih še ne razumete vseh!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Korak 2: Nalaganje vnaprej naučenega modela

Uporabili bomo **MobileNetV2**, nevronsko mrežo, ki je že naučena na milijonih slik.

To se imenuje **Prenos učenja** - uporaba modela, ki ga je nekdo drug že naučil!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Korak 3: Pomožne funkcije

Ustvarimo funkcije za nalaganje in pripravo slik za naš model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Korak 4: Preizkus na vzorčnih slikah

Poskusimo razvrstiti nekaj slik s spleta!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Razvrsti vsako sliko


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Korak 5: Preizkusite svoje slike!

Zamenjajte spodnji URL s katerim koli URL-jem slike, ki jo želite razvrstiti.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Kaj se je pravkar zgodilo?

1. **Naložili smo vnaprej naučen model** - MobileNetV2 je bil naučen na milijonih slik
2. **Predhodno smo obdelali slike** - Spremenili velikost in jih pripravili za model
3. **Model je podal napovedi** - Izpisal je verjetnosti za 1000 objektnih kategorij
4. **Dekodirali smo rezultate** - Številke smo pretvorili v človeku razumljive oznake

### Razumevanje ocene zaupanja

- **90-100%**: Zelo zanesljivo (skoraj zagotovo pravilno)
- **70-90%**: Zanesljivo (verjetno pravilno)
- **50-70%**: Delno zanesljivo (morda pravilno)
- **Pod 50%**: Nezanesljivo (negotovo)

### Zakaj bi lahko bile napovedi napačne?

- **Nenavaden kot ali osvetlitev** - Model je bil naučen na običajnih fotografijah
- **Več objektov** - Model pričakuje en glavni objekt
- **Redki objekti** - Model pozna le 1000 kategorij
- **Slaba kakovost slike** - Zamegljene ali pikselizirane slike so težje prepoznavne

---


## 🚀 Naslednji koraki

1. **Preizkusite različne slike:**
   - Poiščite slike na [Unsplash](https://unsplash.com)
   - Desni klik → "Kopiraj naslov slike" za pridobitev URL-ja

2. **Eksperimentirajte:**
   - Kaj se zgodi z abstraktno umetnostjo?
   - Ali lahko prepozna predmete iz različnih kotov?
   - Kako se spopada z več predmeti naenkrat?

3. **Naučite se več:**
   - Raziščite [lekcije o računalniškem vidu](../lessons/4-ComputerVision/README.md)
   - Naučite se trenirati svoj lasten klasifikator slik
   - Razumite, kako delujejo CNN-ji (konvolucijske nevronske mreže)

---

## 🎉 Čestitke!

Pravkar ste zgradili klasifikator slik z uporabo najsodobnejše nevronske mreže!

Ta ista tehnika poganja:
- Google Photos (organiziranje vaših fotografij)
- Avtomobile brez voznika (prepoznavanje predmetov)
- Medicinsko diagnostiko (analiza rentgenskih slik)
- Nadzor kakovosti (odkrivanje napak)

Nadaljujte z raziskovanjem in učenjem! 🚀



---

**Izjava o omejitvi odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi izhajale iz uporabe tega prevoda.
